In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

# from tqdm import tqdm
from tqdm.notebook import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from open_spiel.python.examples.ubc_cma import *

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy, ActivityPolicy, UndersellPolicy, TiebreakingPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator
import copy
from open_spiel.python.examples.pysats import map_generators, run_sats

from open_spiel.python.examples.sats_game_sampler import test_config_is_wieldy
from open_spiel.python.examples.cfr_utils import read_cfr_config, load_solver
from open_spiel.python.visualizations import ubc_treeviz


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [3]:
def traverse(state, p0, p1):
    if state.is_terminal():
        return
    if state.is_chance_node():
        for action in state.legal_actions():
            traverse(state.child(action), p0, p1)
        return

    istring = state.information_state_string()
    # print(istring)
    try:
        info = policy._infostates[istring][2]
        # print("Visits", info, "Player", state.current_player(), "Strategy Updates", policy._infostates[istring][1].sum().round(2))
    except KeyError:
        return
        # print("0 visits")
 
    
    # print(policy(state))
    
    p = p0 if state.current_player() == 0 else p1
    action_dist = p(state)
    
    # print(action_dist)
        
    # action = max(action_dist, key=action_dist.get) # modal
    # action = list(action_dist.keys())[0] # modal
    
    highest_prob = max(action_dist.values())
    if highest_prob < 0.9:
        print("Mixing?")
        print(action_dist)
        
    for action, prob in action_dist.items():
        if prob > 0:
            return traverse(state.child(action), p0, p1)


In [91]:
# Step 1: Generate games

N_CONFIGS = 1
MIN_TYPES = 2
MAX_TYPES = 2
MIN_BIDDERS = 2
MAX_BIDDERS = 2
MAX_ACTION_SPACE = 9
MAX_NUM_LICENSES = 3
MIN_NUM_LICENSES = 1

configs = []
base = {
    'scale': 1_000_000,
    'auction_params': {
        'increment': .05,
        'max_rounds': 10, # TODO: Think!
        'agent_memory': 10,
        
    },
    'bidders': [
    ]
}
base['auction_params']['agent_memory'] = base['auction_params']['max_rounds']

iters = 0
rng = np.random.default_rng(seed=9995)
failures = defaultdict(int)

with tqdm() as pbar:
    while len(configs) < N_CONFIGS:
        iters += 1
        pbar.update(1)

        sats_seed = rng.integers(int(1e9))
        x = copy.deepcopy(base)
        x['sats_seed'] = sats_seed

        x['map'] = 'OntarioQuebec'

        map_generator, bid_to_quantity_matrix = map_generators[x['map']]
        selected_map = map_generator()
        if bid_to_quantity_matrix is None:
            bid_to_quantity_matrix = np.eye(len(selected_map))

        num_regions, num_products = bid_to_quantity_matrix.shape
        licenses = []
        for _ in range(num_products):
            licenses.append(rng.integers(MIN_NUM_LICENSES, MAX_NUM_LICENSES + 1))

        if np.product([l+1 for l in licenses]) > MAX_ACTION_SPACE:
            continue # This will be too big, just kill it

        if sum(licenses) < 3: # This will not be an interesting game, just kill it
            continue

        ap = x['auction_params']
        ap['licenses'] = licenses

        license_mhz = 10
        mhz_per_pop_open = 0.232 # Real value (for 3800 I think?)

        region_opening_prices = np.array([int(np.round(license_mhz * mhz_per_pop_open * node.population / x['scale'])) for node in selected_map])
        product_opening_prices = region_opening_prices @ bid_to_quantity_matrix # opening prices of encumbered licenses are proportional to bandwidth
        product_opening_prices = np.clip(product_opening_prices, np.min(region_opening_prices) * 0.05, None) # signalling products are worth 5% of cheapest region
        product_opening_prices = np.array([int(np.round(p)) for p in product_opening_prices])
        ap['opening_price'] = product_opening_prices.tolist()
        ap['activity'] = [op for op in ap['opening_price']]

        bidders = x['bidders']
        n_bidders = rng.integers(MIN_BIDDERS, MAX_BIDDERS + 1)

        failed = False
        total_types = 0
        for j in range(n_bidders):
            bidder_types = []
            bidders.append({
                'player': j,
                'types': bidder_types
            })
            for _ in range(rng.integers(MIN_TYPES, MAX_TYPES + 1)):
                bidder = {
                    'type': str(rng.choice(['national', 'regional', 'local'])),
                    'value_per_subscriber': {
                        'lower': 25,
                        'upper': 35,
                    },
                }

                bidder_types.append(bidder)
                total_types += 1
                # TODO: How do I make sure the players are "reasonably" powerful relatively?
                # TODO: Are your types meaningfully different? (Check efficient allocation make sure at least one product for everyone. This check should be first for easier rejection)
                if bidder['type'] == 'regional':
                    bidder['market_share'] = {
                        'lower': 0.04,
                        'upper': 0.1,
                    }
                    bidder['hq'] = rng.integers(len(selected_map))
                elif bidder['type'] == 'local':
                    bidder['market_share'] = {
                        'lower': 0.05,
                        'upper': 0.12,
                    }                       

                    def generate_local_bidder(num_attempts):
                        for _ in range(num_attempts):
                            local_regions = [rng.integers(2) for _ in selected_map] # Can't have no regions or you're values are all 0
                            if 0 < sum(local_regions) < len(local_regions):
                                return local_regions

                    local_regions = generate_local_bidder(500)
                    if local_regions is not None:
                        bidder['local_regions'] = local_regions
                    else:
                        failed = True
                elif bidder['type'] == 'national':
                    bidder['market_share'] = {
                        'lower': 0.08,
                        'upper': 0.15,
                    }
                    if num_regions == 3:
                        bidder['b'] = {
                            'lower': .15,
                            'upper': .5
                        }
                    elif num_regions == 2:
                        bidder['b'] = {
                            'lower': .3,
                            'upper': .9
                        }

                # Want higher marginal values on secondary licenses
                MARKET_SHARE_BOOST = 5 # TODO: Sample this from 1 to 10?
                bidder['market_share']['lower'] = bidder['market_share']['lower'] * MARKET_SHARE_BOOST
                bidder['market_share']['upper'] = bidder['market_share']['upper'] * MARKET_SHARE_BOOST

        
        if total_types < 3:
            continue
                
        if failed:
            continue

        retval = test_config_is_wieldy(x, external=True, test_speed=False)
        if not retval['failed']:
            print("SUCCESS")
            print(x)
            print(retval['sats_config'])
            print(f"Acceptance rate is {len(configs) / iters:.5%}")
            
            
            with tempfile.NamedTemporaryFile(mode='w+') as fp:
                x['auction_params']['heuristic_deviations'] = 0
                sats_config = run_sats(x, fp.name, seed=x['sats_seed'])
                
                retval['sats_config'] = sats_config
                game = pyspiel.load_game('python_clock_auction', dict(filename=fp.name))
                
                cfg = read_cfr_config('cfr_port_9_ext/external_plus_linear') 
                solver = load_solver(cfg, game)
                num_iters = 10_000
                
                # Solve for a bit
                print("Solving...")
                for _ in tqdm_notebook(range(num_iters)):
                    solver.iteration()
                    
                policy = solver.average_policy()
                
                # Does the solver do any substantial mixing?
                traverse(game.new_initial_state(), policy, policy)
            
        else:
            failures[retval['failure_reason']] += 1
            if retval['failure_reason'] == 'Slow MCCFR iters':
                print(failures) 



1371it [01:29, 22.03it/s]

SUCCESS
{'scale': 1000000, 'auction_params': {'increment': 0.05, 'max_rounds': 10, 'agent_memory': 10, 'licenses': [2, 2], 'opening_price': [32, 19], 'activity': [32, 19]}, 'bidders': [{'player': 0, 'types': [{'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 1}, {'type': 'local', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.25, 'upper': 0.6}, 'local_regions': [0, 1]}]}, {'player': 1, 'types': [{'type': 'local', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.25, 'upper': 0.6}, 'local_regions': [1, 0]}, {'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}]}], 'sats_seed': 795488904, 'map': 'OntarioQuebec'}
{'increment': 0.05, 'max_rounds': 10, 'agent_memory': 10, 'licenses': [2, 2], 'opening_price': [32, 19], 'activity': [32, 19], 'license_names': ['Ontario', 'Quebec'], 'pl

/tmp/ipykernel_18286/4170943044.py:166: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(num_iters)):


  0%|          | 0/10000 [00:00<?, ?it/s]

1375it [05:26, 12.53s/it]

Mixing?
{0: 1.9904458586840494e-10, 5: 0.4388899004894792, 8: 0.5611100993114762}
Mixing?
{0: 1.9904458586840494e-10, 5: 0.4388899004894792, 8: 0.5611100993114762}
Mixing?
{0: 6.561679788734605e-11, 7: 0.23141931345108047, 8: 0.7685806864833027}
Mixing?
{0: 6.561679788734605e-11, 7: 0.23141931345108047, 8: 0.7685806864833027}


3401it [07:32,  7.52it/s]

KeyboardInterrupt



In [5]:
# Check on candidate
# candidate = {'scale': 1000000, 'auction_params': {'increment': 0.05, 'max_rounds': 10, 'agent_memory': 10, 'licenses': [3], 'opening_price': [12], 'activity': [12]}, 'bidders': [{'player': 0, 'types': [{'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}, {'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}]}, {'player': 1, 'types': [{'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}, {'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}]}], 'sats_seed': 248007582, 'map': 'BC'}
# candidate = {'scale': 1000000, 'auction_params': {'increment': 0.05, 'max_rounds': 10, 'agent_memory': 10, 'licenses': [4], 'opening_price': [12], 'activity': [12]}, 'bidders': [{'player': 0, 'types': [{'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}, {'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}]}, {'player': 1, 'types': [{'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}, {'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}]}], 'sats_seed': 55913716, 'map': 'BC'}
candidate = {'scale': 1000000, 'auction_params': {'increment': 0.05, 'max_rounds': 10, 'agent_memory': 10, 'licenses': [2, 2], 'opening_price': [32, 19], 'activity': [32, 19]}, 'bidders': [{'player': 0, 'types': [{'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 1}, {'type': 'local', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.25, 'upper': 0.6}, 'local_regions': [0, 1]}]}, {'player': 1, 'types': [{'type': 'local', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.25, 'upper': 0.6}, 'local_regions': [1, 0]}, {'type': 'regional', 'value_per_subscriber': {'lower': 25, 'upper': 35}, 'market_share': {'lower': 0.2, 'upper': 0.5}, 'hq': 0}]}], 'sats_seed': 795488904, 'map': 'OntarioQuebec'}


with tempfile.NamedTemporaryFile(mode='w+') as fp:
    sats_config = run_sats(candidate, fp.name, seed=candidate['sats_seed'])
    candidate_game = pyspiel.load_game('python_clock_auction', dict(filename=fp.name))

    cfg = read_cfr_config('cfr_port_9_ext/external_plus_linear') 
    candidate_solver = load_solver(cfg, candidate_game)
    num_iters = 25_000

    # Solve for a bit
    print("Solving...")
    for _ in tqdm(range(num_iters)):
        candidate_solver.iteration()

    candidate_policy = candidate_solver.average_policy()

    # Does the solver do any substantial mixing?
    traverse(candidate_game.new_initial_state(), candidate_policy, candidate_policy)
    
    # TODO: Also want to know what NC is


{'solver_type': 'cfr', 'solver': 'mccfr', 'sampling_method': 'external', 'regret_matching_plus': True, 'regret_init': 'all', 'regret_init_strength': 1000, 'linear_averaging': True}
Solving...


  0%|                                      | 4/25000 [00:14<24:24:54,  3.52s/it]

KeyboardInterrupt



In [85]:
def draw_game_tree(game, policy, br_policies=None, fname='treeviz.png', figure_dir='figures/2023-06-27-graphviz-neil'):
    if not os.path.exists(figure_dir):
        !mkdir -p {figure_dir}
    
    
    node_policy_decorator, edge_policy_decorator = ubc_treeviz.make_policy_decorators(policy, br_policies)
    gametree = ubc_treeviz.GameTree(
        game,
        node_decorator=node_policy_decorator,
        edge_decorator=edge_policy_decorator,
        group_infosets=False,
        group_terminal=False,
        group_pubsets=False, 
        target_pubset='*',
        depth_limit=20,
        state_prob_limit=0.01,
        action_prob_limit=0.01, 
        policy=policy,
        br_policies=br_policies,
    )
    
    outfile = os.path.join(figure_dir, fname)
    gametree.draw(outfile, prog='dot')
    print("Game tree saved to file", outfile)
    


In [86]:
draw_game_tree(candidate_game, candidate_policy)

Game tree saved to file figures/2023-06-27-graphviz-neil/treeviz.png


In [87]:
# Test nashconv
nash_conv(candidate_game, candidate_policy, return_only_nash_conv=False, restrict_to_heuristics=False)

_NashConvReturn(nash_conv=0.0019613024958324843, player_improvements=array([0.00062237, 0.00133893]), br_policies=[<open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fde96aacee0>, <open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fde59022f40>])

In [88]:
env_and_policy = make_env_and_policy(candidate_game, dict(cfg))
for agent in env_and_policy.agents:
    agent.policy = candidate_policy
for player in range(game.num_players()):
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
modal_policy = env_and_policy.make_policy()
modal_retval = nash_conv(candidate_game, modal_policy, return_only_nash_conv=False, restrict_to_heuristics=False)
modal_retval

_NashConvReturn(nash_conv=0.04516666666666325, player_improvements=array([0.        , 0.04516667]), br_policies=[<open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fde2c1949d0>, <open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fde0a468fd0>])